In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import BaseCrossValidator
from sklearn.linear_model import Ridge
import time
from sklearn.metrics import make_scorer
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
df = pd.read_excel('Data diolah darin.xlsx')
df

,Backhoe,HP(watt),Norm_MAP,RPM,Age,Load_Factor,Engine_Tier,TEMP[C],Fuel[g/s],NOx[g/s],...,NOx[g/hr],HC[g/hr],CO[g/hr],CO2[g/hr],PM[mg/hr],Nox (g/kl),HC (g/kl),CO (g/kl),CO2 (g/kl),PM (g/kl)
0,1,65621.6,0.000000,833.0,12,0.21,2,22.0,0.430000,0.024090,...,86.724000,25.848000,13.176000,4780.656000,14.4,3730.117500,1111.757728,566.717728,2.056225e+05,0.619362
1,1,65621.6,0.012346,800.0,12,0.21,2,22.0,0.620000,0.030320,...,109.152000,25.668000,16.164000,7008.264000,21.6,4694.776362,1104.015681,695.235681,3.014350e+05,0.929047
2,1,65621.6,0.012346,826.0,12,0.21,2,22.0,0.510000,0.030730,...,110.628000,10.980000,10.584000,5722.920000,21.6,4758.261138,472.264772,455.232272,2.461506e+05,0.929047
3,1,65621.6,0.012346,831.0,12,0.21,2,22.0,0.390522,0.023693,...,85.293664,20.747853,4.825082,4382.553038,21.6,3668.596780,892.393432,207.533355,1.884996e+05,0.929047
4,1,65621.6,0.012346,834.0,12,0.21,2,22.0,0.413024,0.026107,...,93.985262,6.979907,9.880556,4673.503069,21.6,4042.434273,300.215326,424.976193,2.010137e+05,0.929047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37523,8,72332.9,0.975904,1676.0,36,0.21,2,41.0,1.590000,0.058830,...,211.788000,19.908000,28.152000,17957.592000,1800.0,445343.100000,41862.100000,59197.400000,3.776083e+07,3785.000000
37524,8,72332.9,0.975904,1683.0,36,0.21,2,42.0,1.320000,0.059500,...,214.200000,19.800000,30.780000,14905.044000,1656.0,542545.340909,50151.250000,77962.397727,3.775286e+07,4194.468182
37525,8,72332.9,0.975904,1846.0,36,0.21,2,42.0,1.530000,0.059640,...,214.704000,21.744000,33.840000,17349.048000,1872.0,469179.694118,47515.850980,73948.509804,3.791183e+07,4090.768627
37526,8,72332.9,0.987952,1876.0,36,0.21,2,42.0,1.760000,0.056200,...,202.320000,22.356000,34.740000,19948.572000,1980.0,384340.943182,42468.990341,65994.485795,3.789568e+07,3761.343750


In [3]:
df.dropna(inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37527 entries, 0 to 37527
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Backhoe      37527 non-null  int64  
 1   HP(watt)     37527 non-null  float64
 2   Norm_MAP     37527 non-null  float64
 3   RPM          37527 non-null  float64
 4   Age          37527 non-null  int64  
 5   Load_Factor  37527 non-null  float64
 6   Engine_Tier  37527 non-null  int64  
 7   TEMP[C]      37527 non-null  float64
 8   Fuel[g/s]    37527 non-null  float64
 9   NOx[g/s]     37527 non-null  float64
 10  HC[g/s]      37527 non-null  float64
 11  CO[g/s]      37527 non-null  float64
 12  CO2[g/s]     37527 non-null  float64
 13  PM[mg/s]     37527 non-null  float64
 14  Nox (g/l)    37527 non-null  float64
 15  HC (g/l)     37527 non-null  float64
 16  CO (g/l)     37527 non-null  float64
 17  CO2 (g/l)    37527 non-null  float64
 18  PM (g/l)     37527 non-null  float64
 19  Fuel[g/hr

In [5]:
X = df[['Backhoe', 'HP(watt)', 'Norm_MAP', 'RPM', 'Age', 'Load_Factor', 'Engine_Tier']]
y = df['CO[g/s]']

In [6]:
PredictorScaler = StandardScaler()
TargetVarScaler = StandardScaler()

PredictorScalerFit = PredictorScaler.fit(X)
TargetVarScalerFit = TargetVarScaler.fit(y.values.reshape(-1, 1))  # Convert y to numpy array and reshape

X = PredictorScalerFit.transform(X)
y = TargetVarScalerFit.transform(y.values.reshape(-1, 1))  # Convert y to numpy array and reshape

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

In [8]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(26268, 7)
(26268,)
(11259, 7)
(11259,)


GBR MODEL

In [9]:
gbr = GradientBoostingRegressor(loss='squared_error')

In [10]:
gbr.fit(X_train,y_train)
y_pred = gbr.predict(X_test)

In [11]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mape(y_true, y_pred):
    return mean_absolute_percentage_error(y_true, y_pred)

rmse_score = rmse(y_test, y_pred)
mape_score = mape(y_test, y_pred)

print(f'RMSE: {rmse_score:.2f}')
print(f"MAPE: {mape_score:.2f}%")

RMSE: 0.71
MAPE: 2.60%


GRID

In [12]:
params = {
    'learning_rate': np.arange(0.1, 1.0, 0.1),
    'n_estimators': np.arange(1, 51, 5),
    'subsample': np.arange(0.5, 1.0, 0.1),
    'min_samples_split': range(2, 11),
    'min_samples_leaf': range(1, 11),
    'max_depth': range(1, 26)
}

rmse_scorer = make_scorer(mean_squared_error, squared=False)
mape_scorer = make_scorer(mean_absolute_percentage_error)

In [13]:
grid = GridSearchCV(
    estimator=gbr,
    param_grid=params,
    scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
    refit='RMSE', 
    cv=2,  
    verbose=3,
    n_jobs=-1
)

start_time = time.time()
gbr_result = grid.fit(X_train,y_train)

print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
print("Best RMSE:", grid.best_score_)
print("Best MAPE:", grid.cv_results_['mean_test_MAPE'][grid.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Fitting 2 folds for each of 1012500 candidates, totalling 2025000 fits
Best: 1.767951 using {'learning_rate': 0.9, 'max_depth': 21, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 46, 'subsample': 0.5}
Best RMSE: 1.7679512709290626
Best MAPE: 8.991574914950089
Execution time: 57757.544157505035 s


In [14]:
best_grid = grid.best_estimator_
y_pred_grid = best_grid.predict(X_test)
rmse_grid = np.sqrt(mean_squared_error(y_test, y_pred_grid))
mape_grid = mean_absolute_percentage_error(y_test, y_pred_grid)
print(f"RMSE: {rmse_grid:.2f}")
print(f"MAPE: {mape_grid:.2f}%")

RMSE: 1.55
MAPE: 7.52%


In [38]:
# export to excel
pd.DataFrame(grid.cv_results_).to_excel('GBR_GS_Scal_CO.xlsx', index=False)

random search

In [16]:
rs = RandomizedSearchCV(
    gbr,
    params,
    scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
    refit='RMSE',
    n_iter=100,
    cv=5,  
    verbose=0,
    n_jobs=-1
)

start_time = time.time()
rs_result = rs.fit(X_train,y_train)

print("Best: %f using %s" % (rs.best_score_, rs.best_params_))
print("Best RMSE:", rs.best_score_)
print("Best MAPE:", rs.cv_results_['mean_test_MAPE'][rs.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Best: 1.341086 using {'subsample': 0.5, 'n_estimators': 21, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_depth': 17, 'learning_rate': 0.9}
Best RMSE: 1.341086498799206
Best MAPE: 7.862634786910357
Execution time: 22.67489457130432 s


In [17]:
best_rs = rs.best_estimator_
y_pred_rs = best_rs.predict(X_test)
rmse_rs = np.sqrt(mean_squared_error(y_test, y_pred_rs))
mape_rs = mean_absolute_percentage_error(y_test, y_pred_rs)
print(f"RMSE: {rmse_rs:.2f}")
print(f"MAPE: {mape_rs:.2f}%")

RMSE: 1.26
MAPE: 6.46%


In [18]:
rs.cv_results_

{'mean_fit_time': array([0.22059975, 1.35688024, 0.08530703, 0.7355041 , 0.69095707,
        0.41489954, 0.59229469, 0.02838101, 0.48672924, 0.23743181,
        0.09823618, 1.57034836, 0.49887667, 0.5560122 , 0.66408219,
        0.1166965 , 1.00716581, 1.83244252, 0.1890099 , 0.03142285,
        1.2152142 , 0.67746277, 0.09347868, 1.0252027 , 2.74413347,
        1.31388688, 1.85813851, 0.63392444, 0.28931203, 0.47188249,
        0.44964509, 0.02450924, 0.16954198, 1.10412402, 0.72450314,
        2.58812013, 0.02704468, 0.27580423, 2.04149904, 0.50009532,
        0.05108943, 0.72181859, 0.24067554, 0.39256716, 1.12085624,
        0.47823477, 2.06093135, 1.14173951, 0.21937737, 0.3265224 ,
        1.79640021, 0.132164  , 1.26656561, 2.5407186 , 0.86998072,
        0.29108162, 2.48081117, 1.56511159, 0.49352694, 2.27887554,
        0.72312775, 0.93266926, 2.37073336, 0.32058873, 1.71319571,
        0.0503366 , 2.26981168, 1.34477639, 1.30005727, 0.50267215,
        0.26098971, 1.06882153,

In [19]:
pd.DataFrame(rs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_depth,param_learning_rate,...,std_test_RMSE,rank_test_RMSE,split0_test_MAPE,split1_test_MAPE,split2_test_MAPE,split3_test_MAPE,split4_test_MAPE,mean_test_MAPE,std_test_MAPE,rank_test_MAPE
0,0.220600,0.018975,0.001244,0.001017,0.9,11,9,7,6,0.4,...,0.091361,91,3.035095,3.320721,2.178299,2.092148,3.218429,2.768938,0.526165,80
1,1.356880,0.118602,0.012568,0.002016,0.8,46,3,4,11,0.9,...,0.072620,21,7.303843,5.072987,4.045672,3.293075,7.457764,5.434668,1.687203,4
2,0.085307,0.007686,0.002745,0.004271,0.5,6,10,10,6,0.1,...,0.109254,36,1.892107,1.562857,1.378891,1.540713,1.861079,1.647129,0.198074,98
3,0.735504,0.085791,0.012303,0.004420,0.7,16,10,5,23,0.7,...,0.087448,26,4.454543,3.584502,3.515590,2.708177,6.079748,4.068512,1.147570,32
4,0.690957,0.085564,0.015548,0.006166,0.6,16,2,1,21,0.1,...,0.085463,86,4.105856,2.067306,2.300470,1.813666,3.984210,2.854302,0.985099,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.043544,0.006505,0.001007,0.000903,0.6,1,7,4,15,0.2,...,0.104057,12,1.266702,1.367166,1.320797,1.013287,1.620383,1.317667,0.194598,99
96,0.105798,0.007268,0.003837,0.005037,0.9,6,4,7,4,0.1,...,0.093669,29,3.345845,1.560090,1.348298,1.498857,1.863412,1.923300,0.730742,93
97,2.024818,0.048823,0.017766,0.002680,0.9,46,5,3,18,0.8,...,0.088874,16,6.435205,4.666238,3.695465,2.798707,7.515980,5.022319,1.734726,8
98,0.284560,0.012852,0.002944,0.003990,0.9,6,8,9,18,0.1,...,0.096829,39,2.442597,1.808616,1.432300,1.218623,2.207200,1.821867,0.458177,96


In [37]:
# export to excel
pd.DataFrame(rs.cv_results_).to_excel('GBR_RS_Scal_CO.xlsx', index=False)

BAYESSIAN

In [28]:
def gb_regression_cv(learning_rate,n_estimators, subsample, min_samples_split, min_samples_leaf, max_depth):
    gbr = GradientBoostingRegressor(
        learning_rate=learning_rate,
        n_estimators=int(n_estimators),
        subsample= (subsample), 
        min_samples_split=int(min_samples_split), 
        min_samples_leaf=int(min_samples_leaf),
        max_depth=int(max_depth)
    )
    scores = cross_val_score(gbr, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse = np.sqrt(-scores.mean())
    return -rmse  # Return only the RMSE score

In [31]:
bayes_opt = BayesianOptimization(
    gb_regression_cv,
    {
        'learning_rate': (0.01, 1.0),
        'n_estimators': (1, 51),
        'subsample': (0.5, 1.0),
        'min_samples_split': (2, 11),
        'min_samples_leaf': (1, 11),
        'max_depth': (1, 26),
    }
)

In [32]:
start_time = time.time()
bayes_opt.maximize(init_points=10, n_iter=100)
print("Execution time: " + str((time.time() - start_time)) + ' s')

|   iter    |  target   | learni... | max_depth | min_sa... | min_sa... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
| 1         | -0.8884   | 0.6451    | 21.74     | 1.394     | 9.803     | 36.48     | 0.6939    |
| 2         | -0.7527   | 0.8267    | 6.827     | 4.57      | 9.816     | 49.75     | 0.9295    |
| 3         | -0.7776   | 0.4922    | 14.41     | 5.22      | 8.241     | 50.86     | 0.8059    |
| 4         | -0.727    | 0.9686    | 2.961     | 9.655     | 10.2      | 23.76     | 0.5897    |
| 5         | -0.6884   | 0.3917    | 24.16     | 9.279     | 8.053     | 6.441     | 0.6989    |
| 6         | -0.7324   | 0.1959    | 18.65     | 3.795     | 2.859     | 44.75     | 0.7276    |
| 7         | -0.7118   | 0.2868    | 21.15     | 3.925     | 10.91     | 11.1      | 0.6273    |
| 8         | -0.7503   | 0.6333    | 15.25     | 9.072     | 6.732     | 29.18     | 0.7724    |
| 9         | -0.733

In [33]:
best_iter = np.argmin([res['target'] for res in bayes_opt.res])

In [34]:
best_params = bayes_opt.res[best_iter]['params']

print("Best Parameters:", best_params)

Best Parameters: {'learning_rate': 0.024792137877421433, 'max_depth': 14.023942485367312, 'min_samples_leaf': 3.5758751050275555, 'min_samples_split': 10.468767391901286, 'n_estimators': 2.0458839744440116, 'subsample': 0.6524984224765533}


In [35]:
best_bs = GradientBoostingRegressor(
    learning_rate=(best_params['learning_rate']),
    n_estimators=int(best_params['n_estimators']),
    subsample=(best_params['subsample']), 
    min_samples_split=int(best_params['min_samples_split']), 
    min_samples_leaf=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']),
    loss='squared_error'
)
best_bs.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.024792137877421433, max_depth=14,
                          min_samples_leaf=2, min_samples_split=10,
                          n_estimators=2, subsample=0.6524984224765533)

In [36]:
y_pred_bs = best_bs.predict(X_test)
rmse_bs = np.sqrt(mean_squared_error(y_test, y_pred_bs))
mape_bs = mean_absolute_percentage_error(y_test, y_pred_bs)

print(f"RMSE: {rmse_bs:.2f}")
print(f"MAPE: {mape_bs:.2f}%")

RMSE: 0.96
MAPE: 1.02%
